In [177]:
import os
import glob
import pandas as pd
import requests
from bs4 import BeautifulSoup
import string
import re
import nltk
nltk.download('cmudict')
cmudict = nltk.corpus.cmudict.dict()

list_positive_scores = []
list_negative_scores = []
list_polarity_scores = []
list_subjectivity_scores = []
list_avg_sentence_length = []
list_percentage_of_complex_words = []
list_fog_index = []
list_avg_words_per_sentence = []
list_total_complex_words_count = []
list_total_words_count = []
list_syllable_per_word = []
list_personal_pronounce = []
list_avg_word_length = []



def count_syllables(word):
    
    # Count the number of syllables in a word using the CMU Pronouncing Dictionary.
    if word not in cmudict:
        return 0
    return max([len([y for y in x if y[-1].isdigit()]) for x in cmudict[word]])


# Remove punctuation marks from the paragraph
translator = str.maketrans('', '', string.punctuation)

# STOPWORDS
# Add the local path to your folder
stopwords_path = 'C:/Users/91942/Desktop/Blackcoffer/Data/StopWords'

# get a list of all text files in the folder
stopwords_files_list = glob.glob(os.path.join(stopwords_path, '*.txt'))

# loop through each file and read its contents
allstopwords = ''
for stopwords_file in stopwords_files_list:
    with open(stopwords_file, 'r') as stopwords:
        allstopwords = allstopwords + ' ' + stopwords.read()
        # do something with the file contents
        
# Remove punctuation marks from the allstopwords
allstopwords = allstopwords.translate(translator)

allstopwords = allstopwords.lower()
allstopwords = allstopwords.replace("\n"," ")
allstopwords = allstopwords.split(" ")
while " " in allstopwords:
    allstopwords.remove('')

# The Master Dictionary (found in the folder MasterDictionary) is used for creating a dictionary of Positive and Negative words.
with open('C:/Users/91942/Desktop/Blackcoffer/Data/MasterDictionary/positive-words.txt', 'r') as file:
    positive_words = file.read()
positive_words = positive_words.split('\n')

# The Master Dictionary (found in the folder MasterDictionary) is used for creating a dictionary of Positive and Negative words.
with open('C:/Users/91942/Desktop/Blackcoffer/Data/MasterDictionary/negative-words.txt', 'r') as file:
    negative_words = file.read()
negative_words = negative_words.split('\n')


input = pd.read_excel("Data/input.xlsx")

for index in range(114):
    page = ''
    page = requests.get(input.iloc[index][1]).text
    
    soup = BeautifulSoup(page,'lxml')
    
    title = soup.find('title').text
    
    div = soup.find_all('div', class_='td-post-content')
    
    ptags = 0
    if len(div) != 0: # index 7
        ptags = len(div[0].find_all('p'))
    
    art_text = ''
    for j in range(ptags):
        art_text = art_text + ' ' + div[0].find_all('p')[j].text
        
    article = title + art_text
    
    article = article.replace('₹', 'Rs.') # index 47
    article = article.replace('≈', '~') # index 80
    article = article.replace("\xa0","")
    article = article.replace("’","'")
    
    filename = input.iloc[index][1].replace(':','%3A')
    filename = filename.replace('/','%2F')
    filename = filename + '.txt'
    
    f = open(filename,"x")
    f.write(article)
    f.close()
    
    article_lower = article.lower()
    
    # Remove punctuation marks from the paragraph
    clean_article = article_lower.translate(translator)

    clean_article_list = clean_article.split(' ')
    while '' in clean_article_list:
        clean_article_list.remove('')
        
    '''
    The Stop Words Lists (found in the folder StopWords) are used to clean the text
    so that Sentiment Analysis can be performed by excluding the words found in Stop Words List.
    '''
    j = -1
    no_stopwords_list = clean_article_list.copy()
    for i in range(len(no_stopwords_list)):
        if no_stopwords_list[j] in allstopwords:
            no_stopwords_list.pop(j)
        else:
            j = j -1
    
    # Positive Score:
    #    This score is calculated by assigning the value of +1 for each word if found in the Positive Dictionary and then adding up all the values.
    positive_score = 0
    for i in no_stopwords_list:
        if i in positive_words:
            positive_score += 1
    list_positive_scores.append(positive_score)

    # Negative Score:
    #    This score is calculated by assigning the value of +1 for each word if found in the Negative Dictionary and then adding up all the values.
    negative_score = 0
    for i in no_stopwords_list:
        if i in negative_words:
            negative_score += 1
    list_negative_scores.append(negative_score)
    
    
    '''
    Polarity Score: This is the score that determines if a given text is positive or negative in nature.
                    It is calculated by using the formula: 
    Polarity Score = (Positive Score – Negative Score)/ ((Positive Score + Negative Score) + 0.000001)
                    Range is from -1 to +1

    '''
    polarity_score = (positive_score - negative_score)/ ((positive_score + negative_score) + 0.000001)
    list_polarity_scores.append(polarity_score)
    
    
    '''
    Subjectivity Score: This is the score that determines if a given text is objective or subjective.
                        It is calculated by using the formula: 
    Subjectivity Score = (Positive Score + Negative Score)/ ((Total Words after cleaning) + 0.000001)
                        Range is from 0 to +1

    '''
    subjectivity_score = (positive_score + negative_score)/ (len(no_stopwords_list) + 0.000001)
    list_subjectivity_scores.append(subjectivity_score)
        
    sentences  = nltk.sent_tokenize(article)
    
    words = len(clean_article_list)
    list_total_words_count.append(words)
    
    avg_words_per_sentence = words / len(sentences)
    # Average Number of Words Per Sentence = the total number of words / the total number of sentences
    list_avg_words_per_sentence.append(avg_words_per_sentence)
    # Average Sentence Length = the number of words / the number of sentences
    list_avg_sentence_length.append(avg_words_per_sentence)
    
    # Complex words are words in the text that contain more than two syllables.
    complex_words = 0
    total_syllables = 0
    syllables = 0

    for j in clean_article_list:
        syllables = count_syllables(j)
        if j[-2:] == 'es' or j[-2:] == 'ed': # i.e. "disconnected"
            if (syllables) > 1:
                esed = count_syllables(j[:-2])
                if (syllables - esed) == 1:
                    syllables = esed
        total_syllables += syllables
        if (syllables) > 2:
            complex_words += 1
    list_total_complex_words_count.append(complex_words)

    
    syllables_per_word = total_syllables / len(clean_article_list)
    list_syllable_per_word.append(syllables_per_word)
    
    # Percentage of Complex words = the number of complex words / the number of words 
    percent_of_complex_words = complex_words / len(clean_article_list)
    list_percentage_of_complex_words.append(percent_of_complex_words)
    
    # Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)
    fog_index = 0.4 * (avg_words_per_sentence + percent_of_complex_words)
    list_fog_index.append(fog_index)
    
    # Define the regular expression pattern to match personal pronouns
    pattern = r'\b(I|me|my|mine|we|us|our|ours|Me|My|Mine|We|Us|Our|Ours)\b'

    # Find all matches of the pattern in the article
    personal_pronouns_list = re.findall(pattern, article)
    personal_pronouns_count = len(personal_pronouns_list)
    list_personal_pronounce.append(personal_pronouns_count)
    
    char_count = 0
    for i in clean_article_list:
        for j in i:
            char_count += 1
    list_avg_word_length.append(char_count / words)
    
    if index % 10 == 0:
        print(index)


df = pd.DataFrame({'POSITIVE SCORE': list_positive_scores,
                   'NEGATIVE SCORE': list_negative_scores,
                   'POLARITY SCORE': list_polarity_scores,
                   'SUBJECTIVITY SCORE': list_subjectivity_scores,
                   'AVG SENTENCE LENGTH': list_avg_sentence_length,
                   'PERCENTAGE OF COMPLEX WORDS': list_percentage_of_complex_words,
                   'FOG INDEX': list_fog_index,
                   'AVG NUMBER OF WORDS PER SENTENCE': list_avg_words_per_sentence,
                   'COMPLEX WORD COUNT': list_total_complex_words_count,
                   'WORD COUNT': list_total_words_count,
                   'SYLLABLE PER WORD': list_syllable_per_word,
                   'PERSONAL PRONOUNS': list_personal_pronounce,
                   'AVG WORD LENGTH': list_avg_word_length})

output = pd.concat([input, df], axis=1)
output.to_csv('C:/Users/91942/Desktop/Blackcoffer/Data/Output Data Structure.csv')
output.to_excel('C:/Users/91942/Desktop/Blackcoffer/Data/Output Data Structure.xlsx')

[nltk_data] Downloading package cmudict to
[nltk_data]     C:\Users\91942\AppData\Roaming\nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


0
10
20
30
40
50
60
70
80
90
100
110


In [171]:
df = pd.DataFrame({'POSITIVE SCORE': list_positive_scores,
                   'NEGATIVE SCORE': list_negative_scores,
                   'POLARITY SCORE': list_polarity_scores,
                   'SUBJECTIVITY SCORE': list_subjectivity_scores,
                   'AVG SENTENCE LENGTH': list_avg_sentence_length,
                   'PERCENTAGE OF COMPLEX WORDS': list_percentage_of_complex_words,
                   'FOG INDEX': list_fog_index,
                   'AVG NUMBER OF WORDS PER SENTENCE': list_avg_words_per_sentence,
                   'COMPLEX WORD COUNT': list_total_complex_words_count,
                   'WORD COUNT': list_total_words_count,
                   'SYLLABLE PER WORD': list_syllable_per_word,
                   'PERSONAL PRONOUNS': list_personal_pronounce,
                   'AVG WORD LENGTH': list_avg_word_length})

df

,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,62,32,0.319149,0.098739,23.760000,0.219978,9.591991,23.760000,392,1782,1.775533,4,5.670595
1,58,37,0.221053,0.174312,17.625000,0.145390,7.108156,17.625000,205,1410,1.560993,8,4.842553
2,64,35,0.292929,0.123441,19.952941,0.207547,8.064195,19.952941,352,1696,1.761203,3,5.400943
3,59,27,0.372093,0.140753,17.391304,0.151250,7.017022,17.391304,242,1600,1.593750,22,4.805625
4,53,23,0.394737,0.102426,22.077922,0.171176,8.899639,22.077922,291,1700,1.604706,20,5.078824
5,41,22,0.301587,0.121622,22.527273,0.155771,9.073217,22.527273,193,1239,1.597256,36,5.003228
6,22,11,0.333333,0.100610,16.377778,0.166893,6.617868,16.377778,123,737,1.620081,11,5.065129
7,0,0,0.000000,0.000000,5.000000,0.000000,2.000000,5.000000,0,5,1.000000,0,6.200000
8,32,13,0.422222,0.150000,19.638889,0.117397,7.902515,19.638889,83,707,1.463932,3,4.830269
9,61,33,0.297872,0.103638,26.604938,0.169838,10.709910,26.604938,366,2155,1.587007,12,4.939675


In [170]:
clean_article_list

['how',
 'to',
 'protect',
 'future',
 'data',
 'and',
 'its',
 'privacy',
 'blackcoffer',
 'insights',
 'before',
 'the',
 'internet',
 'information',
 'was',
 'in',
 'some',
 'ways',
 'restricted',
 'and',
 'more',
 'centralized',
 'the',
 'only',
 'mediums',
 'of',
 'information',
 'were',
 'books',
 'newspapers',
 'and',
 'word',
 'of',
 'mouth',
 'etc',
 'but',
 'now',
 'with',
 'the',
 'advent',
 'of',
 'the',
 'internet',
 'and',
 'improvements',
 'to',
 'computer',
 'technology',
 'moores',
 'law',
 'information',
 'and',
 'data',
 'skyrocketed',
 'and',
 'it',
 'has',
 'become',
 'this',
 'opensystem',
 'where',
 'information',
 'can',
 'be',
 'distributed',
 'to',
 'people',
 'without',
 'any',
 'kind',
 'of',
 'limits',
 'various',
 'publicly',
 'available',
 'tools',
 'have',
 'taken',
 'the',
 'rocket',
 'science',
 'out',
 'of',
 'encrypting',
 'and',
 'decrypting',
 'email',
 'and',
 'files',
 'data',
 'encryption',
 'isnt',
 'just',
 'for',
 'technology',
 'geeks',
 'mo

In [150]:
input

,URL_ID,URL
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...
1,38.0,https://insights.blackcoffer.com/what-if-the-c...
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...
3,40.0,https://insights.blackcoffer.com/will-machine-...
4,41.0,https://insights.blackcoffer.com/will-ai-repla...
...,...,...
109,146.0,https://insights.blackcoffer.com/blockchain-fo...
110,147.0,https://insights.blackcoffer.com/the-future-of...
111,148.0,https://insights.blackcoffer.com/big-data-anal...
112,149.0,https://insights.blackcoffer.com/business-anal...


In [172]:
output = pd.concat([input, df], axis=1)

In [173]:
output

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...,62.0,32.0,0.319149,0.098739,23.760000,0.219978,9.591991,23.760000,392.0,1782.0,1.775533,4.0,5.670595
1,38.0,https://insights.blackcoffer.com/what-if-the-c...,58.0,37.0,0.221053,0.174312,17.625000,0.145390,7.108156,17.625000,205.0,1410.0,1.560993,8.0,4.842553
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...,64.0,35.0,0.292929,0.123441,19.952941,0.207547,8.064195,19.952941,352.0,1696.0,1.761203,3.0,5.400943
3,40.0,https://insights.blackcoffer.com/will-machine-...,59.0,27.0,0.372093,0.140753,17.391304,0.151250,7.017022,17.391304,242.0,1600.0,1.593750,22.0,4.805625
4,41.0,https://insights.blackcoffer.com/will-ai-repla...,53.0,23.0,0.394737,0.102426,22.077922,0.171176,8.899639,22.077922,291.0,1700.0,1.604706,20.0,5.078824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,146.0,https://insights.blackcoffer.com/blockchain-fo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,147.0,https://insights.blackcoffer.com/the-future-of...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,148.0,https://insights.blackcoffer.com/big-data-anal...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112,149.0,https://insights.blackcoffer.com/business-anal...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [174]:
output.to_csv('C:/Users/91942/Desktop/Blackcoffer/Data/Output Data Structure.csv')

In [175]:
output.to_excel('C:/Users/91942/Desktop/Blackcoffer/Data/Output Data Structure.xlsx')